# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
from IPython.display import HTML
import gmaps.datasets
import googlemaps
from ipywidgets.embed import embed_minimal_html
!jupyter nbextension enable --py --sys-prefix gmaps
# Import API key
from api_keys import g_key



Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv("../WeatherPy/cities.csv", index_col=False)
cities_df

,Max Temp,Cloudiness,Humidity,Country,Wind Speed,Lng,City,Lat,Date,parsed_time
0,50.00,90,81,AR,28.86,-68.30,Ushuaia,-54.80,1580002384,2020-01-26 01:33:04
1,43.00,75,60,JP,2.24,139.00,Tochio,37.47,1580002385,2020-01-26 01:33:05
2,79.77,100,82,NC,21.39,164.02,Poum,-20.23,1580002386,2020-01-26 01:33:06
3,66.49,57,52,NZ,8.52,168.33,Bluff,-46.60,1580002387,2020-01-26 01:33:07
4,79.11,0,77,PF,10.22,-134.97,Rikitea,-23.12,1580002388,2020-01-26 01:33:08
...,...,...,...,...,...,...,...,...,...,...
568,48.15,37,49,BO,5.77,-66.23,Atocha,-20.93,1580003063,2020-01-26 01:44:23
569,84.20,75,89,MY,4.70,114.01,Marabu,4.41,1580003065,2020-01-26 01:44:25
570,66.99,43,30,CF,6.13,22.56,Ouango,4.31,1580003067,2020-01-26 01:44:27
571,15.75,0,73,GE,4.92,43.44,Oni,42.58,1580003069,2020-01-26 01:44:29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(api_key=g_key)

figure_layout = {
'width': '800px',
'height': '600px',
'border': '1px solid black',
'padding': '1px'
}
#
fig = gmaps.figure(layout=figure_layout, zoom_level=1.6, center=[41.9028,12.4964])
heatmap_layer = gmaps.heatmap_layer(
    cities_df[['Lat', 'Lng']], weights=cities_df['Humidity'],
    max_intensity=30, point_radius=20.0)
fig.add_layer(heatmap_layer)
fig
#embed_minimal_html('map.html', views=[fig])
#HTML(filename="./map.html")

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Favorable weather conditions for summer vacation: 

* A max temperature lower than 80 degrees but higher than 70.

* Wind speed less than 10 mph.

* Zero cloudiness.


In [11]:
#null and nan variable were not included in this database from the beginning.

cloudiness_mask = cities_df["Cloudiness"] <= 10
windSpeed_mask = cities_df['Wind Speed'] < 10
maxTemp_mask = (cities_df['Max Temp'] > 70) & (cities_df['Max Temp'] < 80)


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
narrowed_city_df = cities_df[cloudiness_mask & windSpeed_mask & maxTemp_mask]
hotel_df = pd.DataFrame()

for index, row in narrowed_city_df.iterrows():
    try:
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={narrowed_city_df.loc[index, 'Lat']},{narrowed_city_df.loc[index,'Lng']}&radius=6000&type=lodging&keyword=hotel&key={g_key}"
        response = requests.get(url).json()
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        hotel_df.loc[index, 'Lat'] = response['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = response['results'][0]['geometry']['location']['lng']
        hotel_df.loc[index, 'City'] = row['City']
        hotel_df.loc[index, 'Country'] = row['Country']
        print(f"Successfully fetched information about hotel near {row['City']}")
    except:
        print(f"Information about hotel near {row['City']} could not be fetched")

Successfully fetched information about hotel near Adoni
Successfully fetched information about hotel near Fuerte Olimpo
Successfully fetched information about hotel near Acajutla
Successfully fetched information about hotel near Arraial do Cabo
Successfully fetched information about hotel near Gualeguay
Successfully fetched information about hotel near San Rafael
Successfully fetched information about hotel near Salaga
Successfully fetched information about hotel near Salto
Successfully fetched information about hotel near Omboue
Successfully fetched information about hotel near Mahalapye
Successfully fetched information about hotel near Rio Cuarto


In [13]:
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Capital O 61155 Chekuri Residency,15.618002,77.288467,Adoni,IN
1,Hotel Fuerte Olimpo,-21.040900,-57.870565,Fuerte Olimpo,PY
2,Royal Decameron Salinitas,13.534211,-89.817660,Acajutla,SV
3,Orlanova Hotel,-22.961411,-42.021496,Arraial do Cabo,BR
4,Hotel Jardín,-33.149766,-59.315325,Gualeguay,AR
5,Petit Hotel,-34.607450,-68.328366,San Rafael,AR
6,Maridon Lodge,8.536087,-0.505111,Salaga,GH
7,Hotel Salto Grande,-31.398102,-58.017880,Salto,UY
8,Hotel Olako,-1.568069,9.262056,Omboue,GA
9,Cresta Mahalapye,-23.094309,26.845833,Mahalapye,BW


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template ="""
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Capital O 61155 Chekuri Residency</dd>\n<dt>City</dt><dd>Adoni</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Fuerte Olimpo</dd>\n<dt>City</dt><dd>Fuerte Olimpo</dd>\n<dt>Country</dt><dd>PY</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Royal Decameron Salinitas</dd>\n<dt>City</dt><dd>Acajutla</dd>\n<dt>Country</dt><dd>SV</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Orlanova Hotel</dd>\n<dt>City</dt><dd>Arraial do Cabo</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Jardín</dd>\n<dt>City</dt><dd>Gualeguay</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Petit Hotel</dd>\n<dt>City</dt><dd>San Rafael</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Maridon Lodge</dd>\n<dt>City</dt><dd>Salaga</dd>\n<dt>Country</dt><dd>GH</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Salto Grande</dd>\n<dt>City</dt><dd>Salto</dd>\n<dt>Country</dt><dd>UY</dd>\n</dl>\n',
 '\n<dl>

In [15]:
# Add marker layer ontop of heat map


markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display Map
#embed_minimal_html('vacation_map.html', views=[fig])
#HTML(filename="./vacation_map.html")

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='800px'))